## **Informações dos dados:**



*   Frequência: 256Hz para 1 segundo -> 256 épocas para cada paciente
*   8 pessoas no grupo de controle e 8 pessoas no grupo de alcólicos
*   Foram usados 64 eletrodos para as medições
*   Cada paciente foi exposto a um estímulo único (S1) ou duplo estímulo (S2)



#### **Bibliotecas utilizadas**

In [26]:
#Bibliotecas essenciais
import numpy as np
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

#Bibliotecas auxiliares
import os
import random
from tqdm import tqdm

#Visualização
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools
from scipy.stats import mannwhitneyu
import plotly.io as pio

init_notebook_mode(connected=True) ## Iniciar Plotly
seed = 42
random.seed = seed

train_files_path = '/content/drive/MyDrive/EEGVisualization/SMNI_CMI_TRAIN/Train/'

pio.renderers.default = 'colab'
pio.renderers

Renderers configuration
-----------------------
    Default renderer: 'colab'
    Available renderers:
        ['plotly_mimetype', 'jupyterlab', 'nteract', 'vscode',
         'notebook', 'notebook_connected', 'kaggle', 'azure', 'colab',
         'cocalc', 'databricks', 'json', 'png', 'jpeg', 'jpg', 'svg',
         'pdf', 'browser', 'firefox', 'chrome', 'chromium', 'iframe',
         'iframe_connected', 'sphinx_gallery']

## **Pré processamento dos arquivos**

In [27]:
train_files_list = os.listdir(train_files_path)

EEG_data = pd.DataFrame({})

for file in tqdm(train_files_list):
  current_df = pd.read_csv(train_files_path + file)
  EEG_data = EEG_data.append(current_df)

#Remover a coluna não usada
EEG_data = EEG_data.drop(['Unnamed: 0'], axis=1) 

#Remover Vírgula 
EEG_data.loc[EEG_data['matching condition'] == 'S2 nomatch,', 'matching condition'] =  'S2 nomatch' 

100%|██████████| 468/468 [05:42<00:00,  1.36it/s]


In [28]:
EEG_data.head()

,trial number,sensor position,sample num,sensor value,subject identifier,matching condition,channel,name,time
0,37,FP1,0,0.905,a,S2 match,0,co2a0000369,0.000000
1,37,FP1,1,-1.048,a,S2 match,0,co2a0000369,0.003906
2,37,FP1,2,-2.024,a,S2 match,0,co2a0000369,0.007812
3,37,FP1,3,-1.536,a,S2 match,0,co2a0000369,0.011719
4,37,FP1,4,-0.559,a,S2 match,0,co2a0000369,0.015625


###**O que cada coluna representa:**


1.  ```trial number``` é o número da tentativa

2.   ```sensor position``` é a posição do eletrodo inserido na cabeça (sistema internacional 10-20)

3.   ```sample num``` varia de 0 a 255 (época)

4.   ```sensor value``` tensão medida em microvolts

5.   ```subject identifier``` 'a' para alcóolico e 'c' para controle

6.   ```matching condition``` S1 obj apenas um objeto; S2 objetos mostrados idênticos ao S1; S2 nomatch objetos mostrados diferindo de S1

7.   ```channel``` representação inteira da coluna ```sensor position```

8.   ```name``` identificação serial do database

9.   ```time``` tempo que se inicia a época







In [46]:
# Trocar alguns valores 'sensor position' pois não se encaixam no EEG
EEG_data.loc[EEG_data['sensor position'] == 'AF1', 'sensor position'] = 'AF3'
EEG_data.loc[EEG_data['sensor position'] == 'AF2', 'sensor position'] = 'AF4'
EEG_data.loc[EEG_data['sensor position'] == 'PO1', 'sensor position'] = 'PO3'
EEG_data.loc[EEG_data['sensor position'] == 'PO2', 'sensor position'] = 'PO4'

## Remover linhas com os canais X, Y e nd pois não se encaixam em nenhum local
EEG_data = EEG_data[(EEG_data['sensor position'] != 'X') & (EEG_data['sensor position'] != 'Y') & (EEG_data['sensor position'] != 'nd')]

## Visualização dos dados

In [47]:
def sample_data(stimulus, random_id=random.randint(0,7)):
    """Função que junta uma pessoa escolhida aleatoriamente do grupo 'a' com o mesmo do grupo 'c' para o mesmo trial"""
    
    ## Escolher aleatoriamente o código dos pacientes dos alcólicos e controlados
    alcoholic_id = EEG_data['name'][(EEG_data['subject identifier'] == 'a') & 
                                    (EEG_data['matching condition'] == stimulus)].unique()[random_id]
    control_id = EEG_data['name'][(EEG_data['subject identifier'] == 'c') & 
                                  (EEG_data['matching condition'] == stimulus)].unique()[random_id]
    
    ## Pegar a menor tentativa de cada paciente escolhido
    alcoholic_trial_number = EEG_data['trial number'][(EEG_data['name'] == alcoholic_id) & (EEG_data['matching condition'] == stimulus)].min()
    control_trial_number = EEG_data['trial number'][(EEG_data['name'] == control_id) & (EEG_data['matching condition'] == stimulus)].min()

    ## Achar o EEG DF com as devidas especificações
    alcoholic_df = EEG_data[(EEG_data['name'] == alcoholic_id) & (EEG_data['trial number'] == alcoholic_trial_number)]
    control_df = EEG_data[(EEG_data['name'] == control_id) & (EEG_data['trial number'] == control_trial_number)]
    
    return alcoholic_df.append(control_df)

In [48]:
sensor_positions = EEG_data[['sensor position', 'channel']].drop_duplicates().reset_index(drop=True).drop(['channel'], axis=1).reset_index(drop=False).rename(columns={'index':'channel'})['sensor position']
channels = EEG_data[['sensor position', 'channel']].drop_duplicates().reset_index(drop=True).drop(['channel'], axis=1).reset_index(drop=False).rename(columns={'index':'channel'})['channel']

def plot_3dSurface_and_heatmap(stimulus, group, df):
    """Função que analisa dado o estímulo, o grupo e os dados gerais plota um gráfico 3D"""

    if group == 'c':
        group_name = 'Control'
    else:
        group_name = 'Alcoholic'
        
    temp_df = pd.pivot_table(df[['channel', 'sample num', 'sensor value']][(df['subject identifier'] == group) & (df['matching condition'] == stimulus)],
                                          index='channel', columns='sample num', values='sensor value').values.tolist()
    data = [go.Surface(z=temp_df, colorscale='Bluered')]

    layout = go.Layout(
        title='<br>3d Surface and Heatmap of Sensor Values for ' + stimulus + ' Stimulus for ' + group_name + ' Group',
        width=800,
        height=900,
        autosize=False,
        margin=dict(t=0, b=0, l=0, r=0),
        scene=dict(
            xaxis=dict(
                title='Time (sample num)',
                gridcolor='rgb(255, 255, 255)',
                showbackground=True,
                backgroundcolor='rgb(230, 230,230)'
            ),
            yaxis=dict(
                title='Channel',
                tickvals=channels,
                ticktext=sensor_positions,
                gridcolor='rgb(255, 255, 255)',
                zerolinecolor='rgb(255, 255, 255)',
                showbackground=True,
                backgroundcolor='rgb(230, 230, 230)'
            ),
            zaxis=dict(
                title='Sensor Value',
                gridcolor='rgb(255, 255, 255)',
                zerolinecolor='rgb(255, 255, 255)',
                showbackground=True,
                backgroundcolor='rgb(230, 230,230)'
            ),
            aspectratio = dict(x=1, y=1, z=0.5),
            aspectmode = 'manual'
        )
    )

    updatemenus=list([
        dict(
            buttons=list([   
                dict(
                    args=['type', 'surface'],
                    label='3D Surface',
                    method='restyle'
                ),
                dict(
                    args=['type', 'heatmap'],
                    label='Heatmap',
                    method='restyle'
                )             
            ]),
            direction = 'left',
            pad = {'r': 10, 't': 10},
            showactive = True,
            type = 'buttons',
            x = 0.1,
            xanchor = 'left',
            y = 1.1,
            yanchor = 'top' 
        ),
    ])

    annotations = list([
        dict(text='Trace type:', x=0, y=1.085, yref='paper', align='left', showarrow=False)
    ])
    layout['updatemenus'] = updatemenus
    layout['annotations'] = annotations

    fig = dict(data=data, layout=layout)
    iplot(fig)

In [49]:
stimulus = 'S1 obj'
S1_sample_df = sample_data(stimulus=stimulus, random_id=1)

### Analisando grupos com apenas um estímulo




In [50]:
#Mostrar gráfico para grupo alcólico com estímulo S1
plot_3dSurface_and_heatmap(stimulus=stimulus, group='a', df=S1_sample_df)

In [51]:
#Mostrar gráfico para grupo controle com estímulo S1
plot_3dSurface_and_heatmap(stimulus=stimulus, group='c', df=S1_sample_df)

### Analisando grupos com dois estímulos match


In [52]:
stimulus = 'S2 match'
S2_m_sample_df = sample_data(stimulus=stimulus, random_id=1)

In [53]:
#Mostrar gráfico para grupo alcólico com estímulo S2 match
plot_3dSurface_and_heatmap(stimulus=stimulus, group='a', df=S2_m_sample_df)

In [54]:
#Mostrar gráfico para grupo controle com estímulo S2 match
plot_3dSurface_and_heatmap(stimulus=stimulus, group='c', df=S2_m_sample_df)

### Analisando grupos com dois estímulos match

In [56]:
stimulus = 'S2 nomatch'
S2_nm_sample_df = sample_data(stimulus=stimulus, random_id=1)

In [57]:
#Mostrar gráfico para grupo alcólico com estímulo S2 nomatch
plot_3dSurface_and_heatmap(stimulus=stimulus, group='a', df=S2_nm_sample_df)

#Note que no gráfico abaixo temos picos na faixa de 80 microvoltz, pelos canais (FP2, AF8, FPZ) 
#podemos perceber que no momento de medição o paciente piscou o olho direito

In [58]:
#Mostrar gráfico para grupo controle com estímulo S2 nomatch
plot_3dSurface_and_heatmap(stimulus=stimulus, group='c', df=S2_nm_sample_df)